In [1]:
import pandas as pd
import pipes as pp
from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline
import numpy as np
from sklearn.metrics import confusion_matrix, precision_score, \
              recall_score, f1_score, precision_recall_curve, roc_curve, roc_auc_score, \
              auc, mean_squared_error
from sklearn.metrics import confusion_matrix, precision_score, \
              recall_score, f1_score, precision_recall_curve, roc_curve, roc_auc_score, \
              auc, mean_squared_error, accuracy_score

# 1.We review the information. Preparing data for processing format

In [2]:
df_ = pd.read_csv('NewDataFrame.csv')
df_.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
df_.drop(["a"], axis=1, inplace=True)

In [3]:
df_ = pp.myPipe.transform(df_)

In [5]:
X = df_.drop(['CreditScore','PaymentBehaviour'], axis = 1)

In [6]:
Y = df_['CreditScore']

# scaling and standardization of values

In [8]:
from sklearn.preprocessing import StandardScaler
mms = StandardScaler()
X = mms.fit_transform(X)

#from sklearn.preprocessing import MinMaxScaler
#mms = MinMaxScaler()
#X = mms.fit_transform(X)

# Создание проверочного набора (Create a validation set)

In [11]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train,  Y_test = train_test_split(X, Y, test_size = 0.5, random_state = 15)

In [12]:
X_train.shape, Y_train.shape

((50000, 43), (50000,))

In [43]:
#X_train.shape, Y_train.shape

((55932, 43), (55932,))

# a. ML With Teacher

# Classification with voting

# Hard Voting

In [44]:
#X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=0)

In [13]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

#Support vector machines (SVMs) 
#are a set of supervised learning methods used for classification, regression and outliers detection.


# Жесткое голосование
#Hard Voting
log_reg = LogisticRegression(random_state = 15)
rnd_clf = RandomForestClassifier(random_state = 15)
svm_clf = SVC(random_state = 15)

voting_clf = VotingClassifier(
    estimators=[('lr', log_reg), ('rf', rnd_clf), ('svc', svm_clf)],
    voting='hard')

In [14]:
voting_clf.fit(X_train, Y_train)

VotingClassifier(estimators=[('lr', LogisticRegression(random_state=0)),
                             ('rf', RandomForestClassifier(random_state=0)),
                             ('svc', SVC(random_state=0))])

In [15]:
#формируем кортеж из всех наших классификаторов, потом их перебираем,тренируем их,делаем предсказание на тестовом наборе и печататем
#we form a tuple of all our classifiers , then iterate over them, train them, make a prediction on the test set and print them
for clf in (log_reg, rnd_clf, svm_clf, voting_clf):
    clf.fit(X_train, Y_train)
    y_pred = clf.predict(X_test)
print(clf.__class__.__name__, accuracy_score(Y_test, y_pred))

KeyboardInterrupt: 

In [ ]:
# Мягкое голосование,считаются вероятности и потом складываются. НО: ВАЖНО,регрессоры должны уметь выдавать вероятность
#не все классификаторы умеют это делать.

# Soft voting, probabilities are considered and then added up. BUT: IMPORTANT, regressors must be able to output the probability
# not all classifiers can do this.

log_reg = LogisticRegression()
rnd_clf = RandomForestClassifier()
svm_clf = SVC(probability=True)

voting_clf = VotingClassifier(
    estimators=[('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)],
    voting='soft')
voting_clf.fit(X_train, Y_train)

In [ ]:
for clf in (log_reg, rnd_clf, svm_clf, voting_clf):
    clf.fit(X_train, Y_train)
    y_pred = clf.predict(X_test)
print(clf.__class__.__name__, accuracy_score(Y_test, y_pred))

# Bagging

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=15)

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag_clf = BaggingClassifier(
    DecisionTreeClassifier(random_state=15), n_estimators=500,
    max_samples=100, bootstrap=True, n_jobs=-1, random_state=15)
bag_clf.fit(X_train, Y_train)
y_pred = bag_clf.predict(X_test)

In [ ]:
print(accuracy_score(Y_test, y_pred))

In [ ]:
tree_clf = DecisionTreeClassifier(random_state=0)
tree_clf.fit(X_train, Y_train)
y_pred_tree = tree_clf.predict(X_test)
print(accuracy_score(Y_test, y_pred_tree))